## Facetted subgrids - Implementation

This notebook is about implementation of the algorithm sketched out in [facet-subgrid.ipynb](facet-subgrid.ipynb).

In [ ]:
%matplotlib inline

from matplotlib import pylab
import matplotlib.patches as patches
import matplotlib.path as path

from ipywidgets import interact, interact_manual
import numpy
import sys
import random
import itertools
import time
import scipy.special
import math
import tikzplotlib

sys.path.append('../..')
from crocodile.synthesis import *
from util.visualize import *

# Helper for marking ranges in a graph
def mark_range(lbl, x0, x1=None, y0=None, y1=None, ax=None):
    if ax is None: ax = pylab.gca()
    if y0 is None: y0 = ax.get_ylim()[1]
    if y1 is None: y1 = ax.get_ylim()[0]
    wdt = ax.get_xlim()[1] - ax.get_xlim()[0]
    ax.add_patch(patches.PathPatch(patches.Path([(x0,y0), (x0,y1)]), linestyle="--"))
    if x1 is not None:
        ax.add_patch(patches.PathPatch(patches.Path([(x1,y0), (x1,y1)]), linestyle="--"))
    else:
        x1 = x0
    if pylab.gca().get_yscale() == 'linear':
        lbl_y = (y0*7+y1) / 8
    else: # Some type of log scale
        lbl_y = (y0**7*y1)**(1/8)
    ax.annotate(lbl, (x1+wdt/200, lbl_y))

def mk_tikz(out):
    tikzplotlib.save(out, axis_height='3.5cm', axis_width='.45\\textwidth', textsize=5,
                     show_info=False, externalize_tables=True)

In [ ]:
pylab.rcParams['figure.figsize'] = 16, 4
pylab.rcParams['image.cmap'] = 'viridis'

## Choosing parameters

There are a lot of parameters involved in facet-subgrid recombination, with quite a few trade-offs involved. Yet in the end, it fundamentally boils down to three concerns:

1. Precision from our outputs
2. Granularity of final and intermediate data - e.g. size of facets/subgrids, but also intermediate data
3. Overhead of the recombination - decides the amount of I/O required, also plays into overall efficiency

The way we are going to approach this is to see precision and granularity as given, then work out the solution with the least overhead from there.

This first requires a model for what parameter we need to reach a certain precision. We start with generating a bunch of candidate prolate-spheroidal wave functions that we might use as windowing functions. We are going to use the grid-space support $W$ to characterise them (=$x_n/y_n$ later), as this is what mostly determines our error.

In [ ]:
W_steps = 32
Ws = numpy.arange(4,22,1/W_steps)
res = 1024
alpha = 0
normal = numpy.prod(numpy.arange(2*alpha-1,0,-2, dtype=float)) # double factorial
pswfs = { W: anti_aliasing_function(res, alpha, numpy.pi*W/2).real / normal
          for W in Ws }

In [ ]:
for W in sorted(pswfs):
    pylab.semilogy(coordinates(res)*res, pswfs[W])

### Base errror

Now we figure out the 'base error' for every value of $W$, i.e. the amount of error we would expect to bleed out into the regions masked by $m$. We are going by the "worst case", which happens if:
1. $y_n=\frac 12y_G$ (i.e. facet covers half the image - note that `res = 2yG` below)
2. $x_A=\frac 12x_G$ (i.e. subgrid covers half the grid)
3. $y_\text{source} = y_n$ (i.e. test pattern assumes source exactly at facet border)


In [ ]:
# Oversampling is in grid space!
def test_pattern(ov = 2):
    # Pattern for source at 1/4
    pattern = pad_mid(numpy.tile([1,1j,-1,-1j], ov*res//8), ov*res)
    # Make symmetric
    assert pattern[ov*res//4] == 1
    assert pattern[3*ov*res//4] == 0
    pattern[ov*res//4] = 0.5
    pattern[3*ov*res//4] = 0.5
    return fft(pattern)
def test_pswf(W, ov = 2):
    n = pad_mid(fft(pad_mid(ifft(pswfs[W]), ov*res // 2)), ov*res)
    n[ov*res//4] /= 2
    n[3*ov*res//4] = n[ov*res//4]
    return n
ov = 2
pylab.plot(coordinates(ov*res)*ov*res, test_pattern(ov).real/res/ov);
pylab.plot(coordinates(ov*res)*ov*res, test_pswf(12, ov).real);
mark_range('$y_{source}=.5 y_G$', ov*res//4);
pylab.legend(['$F[AG]$']); pylab.grid(); pylab.show();

In image space, $AG$ is the test source convolved with a sinc corresponding to the $A$ mask.

In [ ]:
ov = 4
FAG = test_pattern(ov)
FA = fft(pad_mid(numpy.tile([1], ov*res//2+1), ov*res))
def base_err_simple(W, show=False):
    n = test_pswf(W, ov)
    nAG = ifft(n * FAG)
    base_err = numpy.sqrt(numpy.mean(numpy.abs(nAG[:res*ov//4 - int(numpy.ceil(W))])**2))
    if show:
        nA = ifft(n*FA)
        pylab.semilogy(coordinates(ov*res), numpy.abs(ifft(n)), label='n');
        pylab.semilogy(coordinates(ov*res), numpy.abs(ifft(FAG)), label='AG');
        pylab.semilogy(coordinates(ov*res), numpy.abs(nA), label='n*A');
        xAxN = 1/4 + numpy.ceil(W) / res / ov
        mark_range('$x_A$', -1/4, 1/4);
        mark_range('$x_A+x_N$', -xAxN, xAxN);
        pylab.grid(); pylab.ylim(numpy.abs(nA[0])/10, 5);
        pylab.legend(); pylab.show();
        print(f'base error W={W}: {base_err}')
    return base_err
err_base = { W : base_err_simple(W, W == 12) for W in Ws }
pylab.axline((Ws[0], err_base[Ws[0]]), (Ws[-1], err_base[Ws[-1]]), c='gray', ls=':')
pylab.semilogy(Ws, [ err_base[W] for W in Ws ]);
pylab.xlim(Ws[0],Ws[-1]); pylab.ylabel('base error'); pylab.xlabel('$W=x_n/y_n$'); pylab.grid();

The base error is given by the average signal strenght left outside the $x_A+x_n$ region. Note that the position of the source in the grid ($G$) indeed makes a difference here: $n*AG$ settles multiple orders of magnitude higher than $n*A$ (which would be equivalent to a source in the centre).

Note that when we vary $W$, the base error evolves basically in a log-linear fashion.

### Facet margins

The base error tells us how much error we have to expect when we actually crop to $x_A+x_n$ size. This error will get amplified once we apply the inverse convolution $b$. As $n$ falls to zero quickly at certain points in image space, $b$ will get very large, amplifying the base error significantly.

This can again be derived directly from the window function - for any target error we just need to figure out how much extra allowance we can afford, then find where the inverse of the convolution reaches that value.

In [ ]:
def find_x_sorted_smooth(xs, ys, y):
    assert(len(xs) == len(ys))
    pos = numpy.searchsorted(ys, y)
    if pos <= 0:
        return xs[0]
    if pos >= len(ys) or ys[pos] == ys[pos-1]:
        return xs[len(ys)-1]
    w = (y - ys[pos-1]) / (ys[pos] - ys[pos-1])
    return xs[pos-1] * (1-w) + xs[pos] * w
def find_x_sorted_logsmooth(xs, ys, y):
    return find_x_sorted_smooth(xs, numpy.log(numpy.maximum(1e-100, ys)), numpy.log(y))

In [ ]:
def find_x0(W, err, dim=1):
    err /= err_base[W]; target = 1/err**(1/dim)
    return find_x_sorted_logsmooth(-coordinates(res)[:res//2], pswfs[W][:res//2], target)
find_x0(12,1e-5) # 0.3632584490529124

In [ ]:
for W in numpy.arange(4, 16):
    err = numpy.exp(-numpy.arange(0, 10, 0.125/4) * numpy.log(10))
    pylab.semilogx(err, [ find_x0(W, e) for e in err ], label="W=%d"%W)
pylab.xlabel('Error'); pylab.ylabel('$y_B/y_n$'); pylab.legend(); pylab.ylim(-0.01, 0.51); pylab.grid();

In [ ]:
for W in numpy.arange(4, 16):
    err = numpy.exp(-numpy.arange(0, 10, 0.125/2) * numpy.log(10))
    pylab.semilogx(err, [ find_x0(W, e, 2) for e in err ], label="W=%d"%W)
pylab.xlabel('Error'); pylab.ylabel('$x_0$ (approx)'); pylab.legend(); pylab.ylim(-0.01, 0.51);
pylab.grid();

### Subgrid margins 1: Decompression 

To enable handling subgrids without the need to solve grid-sized Fourier transforms, the subgrid mask must itself have a representation that has bounds in image space. What we are looking for is exactly the same properties as with the signal windowing function, therefore we simply use the same one.

Theoretically we would reason that $n*AG$ needs $x_A+x_n$ margins, therefore $(\hat{m}*n)(n*AG)$ would require $x_A+2x_n$. However, clearly $n*AG$ is very small close to the $x_A+x_n$ bounds, therefore we can actually choose the bounds somewhat smaller without introducing too much error. As we are essentially multiplying two shifted $n$, we can approximate the required margin by determining the point where $n*n$ (i.e. the window function convolved with itself) reaches the target error. We will call that point $x_{n^2}$.

As this error will appear similarly as the truncation error from $n$ we will aim for the same base error as determined above.

In [ ]:
W2s = []
ov = 2
xA = 0.25
FA = numpy.sinc(coordinates(res*ov)*xA*2*res*ov)*xA*2*res*ov
FAG = test_pattern(ov)
for W in Ws:
    pswf = test_pswf(W,ov)
    xN2 = 2*find_x_sorted_logsmooth(-coordinates(res*ov)[:res*ov//2], numpy.abs(ifft(pswf)**2)[:res*ov//2],
                                    err_base[W])
    if W == 23:
        pylab.subplot(121)
        Fm = numpy.sinc(coordinates(res*ov)*((xA+xN2)*2*res*ov))*((xA+xN2)*2*res*ov)
        Fm1 = -Fm; Fm1[res*ov//2] += res*ov; # i.e. Fm1 = fft(1-ifft(Fm))
        pylab.axline((pylab.gca().get_xlim()[0], err_base[W]), (pylab.gca().get_xlim()[1], err_base[W]),
                     c='black', linestyle=':', linewidth=1)
        pylab.semilogy(coordinates(res*ov), numpy.abs(ifft(Fm*pswf)), label='$\hat m*n$')
        pylab.semilogy(coordinates(res*ov), numpy.abs(ifft(Fm1*pswf)),
                       label='$(1-\hat{m})*n$')
        pylab.semilogy(coordinates(res*ov), numpy.abs(ifft(pswf*FA)), label='$n*A$')
        pylab.semilogy(coordinates(res*ov), numpy.abs(ifft(Fm1*pswf)*ifft(pswf*FA)),
                       label='$((1-\hat m)*n)(n*A)$')
        pylab.semilogy(coordinates(res*ov), numpy.abs(ifft(pswf*FAG)), label='$n*AG$')
        pylab.semilogy(coordinates(res*ov), numpy.abs(ifft(Fm1*pswf)*ifft(pswf*FAG)),
                       label='$((1-\hat m)*n)(n*AG)$')
        xN = W / res / ov
        mark_range('$x_A$', xA); mark_range('$x_A+x_n$', xN+xA); mark_range('$x_A+x_{n^2}$', xN2+xA)
        mark_range('', xN2/2+xA); pylab.xlim(xA-0.05,xA+xN2+0.05); pylab.ylim(err_base[W]**1.3/1e2, 2e0);
        pylab.title(f'subgrid mask border for W={W} (base={err_base[W]:.3g} $x_{{n^2}}/x_n={xN2/xN:.3f}$)');
        pylab.legend(loc='center left'); pylab.grid();
    W2s.append(xN2 * res * ov)
pylab.subplot(122)
pylab.plot(Ws, W2s);
slope = (W2s[-1] - W2s[0]) / (Ws[-1] - Ws[0]) 
print( numpy.average(W2s - slope * Ws), "+W*", slope )
pylab.axline((Ws[0], W2s[0]), (Ws[-1], W2s[-1]), c='black', linestyle=':')
pylab.ylabel('$x_{n^2}/x_n$'); pylab.title('extra border required'); pylab.xlabel('W')
W2s_map = { W: W2 for W, W2 in zip(Ws, W2s)};
mk_tikz('masking.tikz')

Both error terms $((1-\hat{m})*n)(n*A)$ and  $((1-\hat{m})*n)(n*AG)$ (where $G$ is again the test pattern form above) show a clear maximum at $x_A+\frac 12x_{n^2}$. Note that $x_{n^2}/x_n$ depends on $W$, but reaches $\sim 1.6$ at worst.

### Subgrid margins 2: Non-coplanarity

In order to correct for array non-coplanarity, we might want to add allowances for adding Fresnel terms after recombination. Those are given as:

$$G_w(l) = \mathcal F_l\left[ e^{2\pi i w \sqrt{1 - l^2}} \right]$$

Assuming that the maximum derivative of $w \sqrt{1 - l^2}$ is given as $h$ (for hardness), and the maximum $l$ value is given by $theta$ (maximum field of view). 


In [ ]:
@interact(ymax=(0,1,0.05), yN=(0,1,0.05), yG=(0,1,0.1), w=(0,1000, 1), W=(4,22-1/W_steps,1/W_steps))
def test_noncoplanar(ymax=0.1, yN=0.1, yG=0.5, w=600, W=12):
    pylab.clf()
    for iw,W_ in enumerate(Ws):
        if W - W_ < 1e-5:
            break
    W = W_
    pswf = pswfs[W]
    N = int(res*yG/yN); xG = N / yG / 4    
    xN = W / 4 / yN; xN2 = W2s[iw] / 4 / yN; ys = yG*2*coordinates(N)+ymax-yN; xs = xG*2*coordinates(N)   
    xgw = w*ymax/numpy.sqrt(1-ymax**2)
    xgw_mid = w*(ymax-yN)/numpy.sqrt(1-(ymax-yN)**2)
    print('xN=', xN, ' xN2=', xN2, ' x_gw=', xgw, ' N=', N)
    n = pad_mid(pswf, N)
    g = pad_mid(numpy.exp(2j*numpy.pi*w*numpy.sqrt(1-extract_mid(ys, res)**2)), N)
    N0 = int(numpy.ceil(xgw_mid / xG / 2 * N + N // 2))
    xNG = find_x_sorted_logsmooth(xs[:N0:-1], numpy.abs(ifft(g*n))[:N0:-1], err_base[W])
    xNG2 = find_x_sorted_logsmooth(xs[:N0:-1], numpy.abs(ifft(g*n*n))[:N0:-1], err_base[W])
    print('xNG=', xNG, ' xNG2=', xNG2)
    pylab.semilogy(xs,numpy.abs(ifft(n)), label='n')
    pylab.semilogy(xs,numpy.abs(ifft(n*n)), label='n*n')
    pylab.semilogy(xs,numpy.abs(ifft(g)), label='g')
    pylab.semilogy(xs,numpy.abs(ifft(g*n)), label='g*n')
    pylab.semilogy(xs,numpy.abs(ifft(g*n*n)), label='g*n*n')
    pylab.legend()
    pylab.xlim(-max(xNG,xgw)-2*xN, max(xNG,xgw)+2*xN); pylab.ylim(err_base[W]/1e3,1);
    mark_range('$x_{n}$', -xN,xN); mark_range('$x_{n^2}$', -xN2,xN2)
    mark_range('$x_{g}$', -xgw,xgw); mark_range('$x_{ng}$', -xNG,xNG)
    mark_range('$x_{n^2g}$', -xNG2,xNG2)
    pylab.axline((-xG, err_base[W]), (xG, err_base[W]), c='black', linestyle=':', linewidth=1);
    mk_tikz('noncoplanarity.tikz')

In [ ]:
ymax=0.2
yG=0.5
w=600
_Ws = list(reversed(Ws[W_steps*2::W_steps*2]))
for yN in [0.1, 0.2]:
    for iw, W in enumerate(_Ws):
        pswf = pswfs[W]
        N = int(res*yG/yN); xG = N / yG / 4    
        xN = W / 4 / yN; xN2 = W2s[iw] / 4 / yN; ys = yG*2*coordinates(N)+ymax-yN; xs = xG*2*coordinates(N)   
        n = pad_mid(pswf, N)
        xgws = []; xNGs = []; xNG2s = []
        for w in numpy.arange(0, 2000, 10):
            xgw = w*ymax/numpy.sqrt(1-ymax**2)
            xgw_mid = w*(ymax-yN)/numpy.sqrt(1-(ymax-yN)**2)
            g = pad_mid(numpy.exp(2j*numpy.pi*w*numpy.sqrt(1-extract_mid(ys, res)**2)), N)
            N0 = int(numpy.ceil(xgw_mid / xG / 2 * N + N // 2))
            xNG = find_x_sorted_logsmooth(xs[:N0:-1], numpy.abs(ifft(g*n))[:N0:-1], err_base[W])
            xNG2 = find_x_sorted_logsmooth(xs[:N0:-1], numpy.abs(ifft(g*n*n))[:N0:-1], err_base[W])
            xgws.append(xgw); xNGs.append(xNG); xNG2s.append(xNG2)
        pylab.plot(xgws, xNG2s - numpy.array(xgws), color=f'C{iw}', lw=yN*8)
pylab.xlim(0, xgw); pylab.grid(); pylab.legend([f'W={W}' for W in _Ws]);

On the other hand, for choosingthe granularity of the data we are mostly constrained by the desired facet and subgrid size. Except we generally do not actually care *that* much about its exact values, especially for facets: We just want them to be a certain order of magnitude so we can work with them within the memory constraints of a node or a CPU's cache.

The point in our algorithm where we do care a lot about the exact value is whenever we solve a Fast Fourier Transform. These transforms are clearly at their most efficient if the sizes decompose into only small primes. In the ideal case simply a power of 2. As it turns out, we will need to solve FFTs in three sizes:

1. $2y_P$
2. $4x_My_P$
3. $2x_MN$

Therefore we have to only choose these three values appropriately in order to get optimal FFT performance. As we now are closer to the implementation side, let's just start with the sizes we would like to see:

In [ ]:
target_err = 1e-5

N = 1024 
yP_size = 512
xM_size = 256
#N = 128 * 1024
#yP_size = 32 * 1024
#xM_size = 512
fov = 2 * 0.375

assert(xM_size * yP_size % N == 0)

xE = 0 # extra x space, e.g. for w-kernel

assert(N % xM_size == 0)
xM_step = N // xM_size

These are actually all the constraints we need - N tells us the total image size, `xM_size` the rough size of subgrids, and `yP_size` the rough size of facets. No we "just" need to figure out what actual configurations are viable.

First we need to keep in mind that we need margins both in grid and image space to make this work. We need:
$$x_A < x_M - 2x_N$$
So we need to leave enough margin in grid space to fit the gridding function $n$ twice - once for actual convolution, and then again to allow us to short-cut the $m$ mask operation. Furthermore we require:
$$y_B < 2y_P - 2y_N$$
To leave enough room so we don't get aliasing when masking $m$. Also remember that $y_N$ here is $y_B$ padded to make gridding accurate, so with the nomenclature from above:
$$y_N = \frac{y_B}{2x_0} \quad\Rightarrow\quad y_B < \frac{2y_P}{1+ \frac 1{x_0}}$$

To tie everything together, note that $x_N = \frac{R}{y_N}$. From here we can calculate an "ideal" overhead value simply by comparing how much we have lost with $x_A$ compared to $x_M$ (depends on $R$) and $y_B$ compared with $y_N$ (depends on $x_0$):

In [ ]:
def calc_param_bounds(N, yP_size, xM_size, W, err, dim=1):
    #x0 = approx_x0(W/2, err)
    x0 = max(0.001, find_x0(W, err, dim))
    yB_size_m = 2 * yP_size / (1+1/x0)
    yN_size_m = yB_size_m / 2 / x0
    xN_size = W/yN_size_m*N
    xN2_size = W2s_map[W] / yN_size_m * N
    xA_size_m = xM_size - xE - xN2_size
    if xA_size_m > 0:
        base_ov = yN_size_m * xM_size / xA_size_m / yB_size_m
    else:
        base_ov = None
    return x0, xN_size, yN_size_m, xA_size_m, yB_size_m, base_ov

for err in 10.**numpy.arange(-9,-2):
    b_overheads = [ calc_param_bounds(N, yP_size, xM_size, W, err)[-1] for W in Ws ]
    pylab.plot(Ws, b_overheads, label="err=%.0e" % err)
pylab.ylabel("Overhead"); pylab.xlabel("W"); pylab.ylim(1,10); pylab.legend(); pylab.show();
for err in 10.**numpy.arange(-9,-2):
    b_overheads = [ calc_param_bounds(N, yP_size, xM_size, W, err)[0] for W in Ws ]
    pylab.plot(Ws, b_overheads, label="err=%.0e" % err)
pylab.ylabel("$x_0$");pylab.xlabel("W"); pylab.legend(); pylab.ylim(0,0.5); pylab.show();

Looks nice and smooth, but this isn't the full story. To actually identify usable parameters for our algorithm to be efficient we want to apply simple coordinate shifts on down-sampled grids/images in order to position the contributions of subgrids/subimages. This *only* works if the facet and subgrid offsets are actually chosen such that they correspond to whole numbers even after down-sampling.

In practice, this boils down to finding a suitable decomposition $N_x N_y = N$ with $N_x, N_y \in \mathbb{N}$ such that:

1. $N_x$ divides $x_M$ and $x_A$ ($\cdot 2N$)
2. $N_y$ divides $y_B$, $y_N$ and $y_P$ ($\cdot 2$)

The reasons are relatively obvious for the sizes that correspond to the down-sampling we do (specifically $y_N$, $y_P$ and $x_M$).

On the other hand, while $x_A$ and $y_B$ are "just" about the usable area of facets and subgrids, in practice that usable area is only really useful up to the same multiplicators. The reason is that we often want to cover a larger area with subgrids and/or subimages, which leads to a "tiling". However as $N_x$ and $N_y$ restrict the valid subimage / subgrid offsets we can use, we only have certain choices of spacings between those tiles. This means that we can only use subimage/subfacet area up to the greatest multiple or $N_x$/$N_y$ in a systematic way. By insisting on $x_A$ / $y_B$ be divisible we simply include this in our overhead calculation.

Of the parameters given above, we have $N$, $x_M$ and $y_P$ given, so those simply restrict the possible values of $N_x$ and $N_y$. However, we have to identify $N_x$ and $N_y$ such that we get minimum overhead from adjusting $x_A$, $y_B$ and $y_N$ accordingly. It is easiest to simply try all options:

In [ ]:
def find_best_configuration(N, yP_size, xM_size, W, err, wtowers = False, fov = 1):
    """ Find best imaging configuration
    :param N: Total grid/imaeg size
    :param yP_size: Padded facet size
    :param xM_size: Padded subgrid size
    :param W: PSWF parameter
    :param wtowers: Use w-towers efficiency (3D) instead of plain efficiency (1D)?
    :param fov: How how of the field of view is assumed to be required
    :returns: Parameter dictionary
    """
    assert(xM_size * yP_size % N == 0)
    assert(N % xM_size == 0)

    # Get parameter bounds, exit early if impossible
    x0, xN_size, yN_size_m, xA_size_m, yB_size_m, base_eff = calc_param_bounds(N, yP_size, xM_size, W, err, dim=2)
    if xA_size_m <= 0 and yB_size_m <= 0:
        return None

    #print("R=%g, x0=%g, yB_size<%g, yN_size<%g, xA_size<%g, ov=" % (R, x0, yB_size_m, yN_size_m, xA_size_m), end='')
    base_eff = 1 / (yN_size_m * xM_size / xA_size_m / yB_size_m)
    base_eff_g = 1 / ((yN_size_m * xM_size / (xA_size_m * 2 / 3) / yB_size_m)**2 / (xA_size_m / 3))
    best_eff = 1e-6; best_pars = None;
    xM_step = N // xM_size
    for Ny in xM_step * numpy.arange(1, N // xM_step):
        if N % Ny != 0: continue
        Nx = N // Ny
        if Nx > xA_size_m or Ny > yB_size_m: continue
        xA_size = Nx * int(xA_size_m // Nx) # round down
        yB_size = Ny * int(yB_size_m // Ny) # round down
        yN_size = xM_step * int((yN_size_m + xM_step - 1) // xM_step) # round up
        # Rounding might (extremely rarely) cause us to violate the side condition
        while yB_size / 2 + yN_size > yP_size:
            yB_size -= Ny
        if yB_size <= 0:
            continue
        # Determine how many facets we'll need to cover the field of view.
        # By reducing the "effective" facet size we can represent the
        # inefficiency from unused facet space
        if fov is not None:
            nfacet = int(numpy.ceil(N * fov / yB_size))
            if nfacet % 2 == 0: nfacet += 1
            yB_size_eff = N * fov / nfacet
        else:
            yB_size_eff = yB_size
        base_pars = dict(
            N=N, yP_size=yP_size, xM_size=xM_size, W=W, fov=fov,
            x0=x0, Nx=Nx, Ny=Ny, xA_size=xA_size,
            yB_size=yB_size, xN_size=xN_size, yN_size=yN_size
        )
        if not wtowers:
            # Calculate efficiency
            eff = 1 / (xM_size * yN_size / xA_size / yB_size_eff)
            if eff > best_eff:
                best_eff = eff;
                best_pars = dict(eff=eff, base_eff=base_eff, **base_pars)
        else:
            # Calculate uvw efficiency
            best_eff3_g = 1e-15; best_xA_size_g = None
            for j in range(100):
                xA_size_g = xA_size - j * Nx
                if xA_size_g <= 0: break
                if xA_size_g >= xA_size_m: continue
                ov_g = 1 / ( (xM_size * yN_size / xA_size_g / yB_size_eff)**2 / (xA_size_m - xA_size_g))
                if ov_g > best_eff3_g:
                    best_eff3_g = ov_g; best_xA_size_g = xA_size_g
            if best_eff3_g > best_eff:
                best_eff = best_eff3_g
                best_pars = dict(eff=best_eff3_g, base_eff=base_eff_g, xA_size_g=best_xA_size_g, **base_pars)

    return best_pars

In [ ]:
plot1, plot2 = pylab.figure(figsize=(16,8)).subplots(2)

plot1.set_ylabel("$x_Ay_B(x_my_n)^{-1}$")
plot1.set_xlabel("$W = 4x_Ny_N$")
plot2.set_ylabel("$(x_A^2x_g)(y_B^2y_G)(x_my_n)^{-1}$")
plot2.set_xlabel("$W = 4x_Ny_N$")

mk_patch = lambda x, y, c: patches.Patch((x,y),eg=f'C{c}', fc=f'C{c}')


for i, err in enumerate(numpy.exp(numpy.arange(-3,-10,-1)*numpy.log(10))):
    #Rs = numpy.arange(2,15,1/64)
    b_effectives = []; effectives = []; b_effectives_g = []; effectives_g = []
    best_eff = 1e-6; best_pars = None; best_eff_g = 1e-6; best_pars_g = None
    print("err=%g" % err)
    for W in Ws:

        # Find best configuration - with wtowers and without
        best_pars2 = find_best_configuration(N, yP_size, xM_size, W, err, fov=fov)
        best_pars2_g = find_best_configuration(N, yP_size, xM_size, W, err, True, fov=fov)

        if best_pars2 is not None and best_pars2['eff'] > best_eff:
            best_eff = best_pars2['eff']; best_pars = best_pars2
        if best_pars2_g is not None and best_pars2_g['eff'] > best_eff_g:
            best_eff_g = best_pars2_g['eff']; best_pars_g = best_pars2_g
        b_effectives.append(best_pars2['base_eff'] if best_pars2 is not None else 0)
        effectives.append(best_pars2['eff'] if best_pars2 is not None else 0)
        b_effectives_g.append(best_pars2_g['base_eff'] if best_pars2_g is not None else 0)
        effectives_g.append(best_pars2_g['eff'] if best_pars2_g is not None else 0)        
        
    plot1.plot(Ws, b_effectives, linestyle=':', c = f'C{i}')
    plot1.plot(Ws, effectives, label="error %g" % err, c = f'C{i}')
    plot2.plot(Ws, b_effectives_g, linestyle=':', c = f'C{i}')
    plot2.plot(Ws, effectives_g, label="error %g" % err, c = f'C{i}')

    if best_pars is not None:
        print((" => eff={eff:.2f} (<{base_eff:.2f}),\tW={W:.2f}, x0={x0:.2f}, Nx={Nx}, Ny={Ny},"
               " xA_s={xA_size}, yB_s={yB_size}, xN_s={xN_size:.2f}, yN_s={yN_size}").format(**best_pars) )
        plot1.plot(best_pars['W'], best_pars['eff'], c = f'C{i}', marker='o')
    if best_pars_g is not None:
        print((" => eff={eff:.2f} (<{base_eff:.2f}),\tW={W:.2f}, x0={x0:.2f}, Nx={Nx}, Ny={Ny},"
               " xA_s={xA_size_g}/{xA_size}, yB_s={yB_size}, xN_s={xN_size:.2f}, yN_s={yN_size}").format(**best_pars_g) )
        plot2.plot(best_pars_g['W'], best_pars_g['eff'], c = f'C{i}', marker='o')
    if abs(err - target_err) < 1e-12:
        target_pars = best_pars_g
        print("^^^")

plot1.legend()
plot1.set_ylim(1e-6,plot1.get_ylim()[1]); plot1.set_xlim(4, 22)
plot1.grid();
plot2.set_ylim(1e-6,plot2.get_ylim()[1]); plot2.set_xlim(4, 22)
plot2.grid()

for n in target_pars:
    exec(f'{n} = target_pars["{n}"]')
mk_tikz("effective-%dk-%dk-%gk.tikz" % (N // 1024, yP_size // 1024, xM_size / 1024))

In [ ]:
xN = xN_size / 2 / N
xM = xM_size / 2 / N
yN = yN_size / 2
xA = xA_size / 2 / N
yB = yB_size / 2
print("xN=%g xM=%g yN=%g xNyN=%g xA=%g" % (xN, xM, yN, xN*yN, xA))

xM_yP_size = xM_size * yP_size // N
xMxN_yP_size = xM_yP_size + int(2 * numpy.ceil(xN_size * yP_size / N / 2))
assert((xM_size * yN_size) % N == 0)
xM_yN_size = xM_size * yN_size // N
def fmt(x):
    if x >= 1024*1024 and (x % (1024*1024)) == 0:
        return "%dM" % (x // 1024 // 1024)
    if x >= 1024 and (x % 1024) == 0:
        return "%dk" % (x // 1024)
    return "%d" % x
cfg_name = "%s_%s_%s_%g" % (fmt(N), fmt(yP_size), fmt(xM_size), target_err)
print("xM_yP_size=%d, xMxN_yP_size=%d, xM_yN_size=%d" % (xM_yP_size, xMxN_yP_size, xM_yN_size))
if fov is not None:
    nfacet = int(numpy.ceil(N * fov / yB_size))
    print(f"{nfacet}x{nfacet} facets for FoV of {fov} ({N * fov / nfacet / yB_size * 100}% efficiency)")
print("\n%s: %d,%d,%d,%d,%d,%d,%d,%d" % (cfg_name, N, Nx, yB_size, yN_size, yP_size, xA_size, xM_size, xMxN_yP_size))

In [ ]:
def find_best_configuration_W(N, yP_size, xM_size, err, wtowers = False):
    best_eff = 0; best_pars = None
    for W in Ws:
        best_pars2 = find_best_configuration(N, yP_size, xM_size, W, err, wtowers)
        if best_pars2 is not None and best_pars2['eff'] > best_eff:
            best_eff = best_pars2['eff']; best_pars = best_pars2
    return best_pars

for N_, yP_size_, xM_size_ in [
    ( 64 * 1024,  8 * 1024, 512),
    ( 64 * 1024, 16 * 1024, 256),
    ( 96 * 1024, 12 * 1024, 512),
    ( 96 * 1024, 24 * 1024, 256),
    (128 * 1024, 16 * 1024, 512),
    (128 * 1024, 32 * 1024, 256),
    (256 * 1024, 32 * 1024, 512),
    (512 * 1024, 64 * 1024, 512),
    ( 64 * 1024, 12 * 1024, 512),
    ( 64 * 1024, 24 * 1024, 256),
    (128 * 1024, 24 * 1024, 512),
    (128 * 1024, 48 * 1024, 256),
    (256 * 1024, 48 * 1024, 512),
    
    #( 64 * 1024,  8 * 1024, 512),
    #(128 * 1024, 16 * 1024, 512),
    #(128 * 1024, 32 * 1024, 256),
    #(256 * 1024, 32 * 1024, 512),    
]:
    best_pars = find_best_configuration_W(N_, yP_size_, xM_size_, err, True)
    best_pars['eff'] *= 100
    print('{N:,} & {yP_size:,} & {yN_size:,} & {yB_size:,} & {xM_size:,} & {xA_size:,} & {xA_size_g:,} & {eff:.1f}\% \\'
          .format(**best_pars).replace(',','\,'))

## Calculate actual PSWF to use

Same as above, but this time we calculate it to the full required resolution (facet size)

In [ ]:
alpha = 0
pswf = anti_aliasing_function(yN_size, alpha, numpy.pi*2*xN*yN).real
pswf /= numpy.prod(numpy.arange(2*alpha-1,0,-2, dtype=float)) # double factorial

Check that PSWF indeed satisfies intended bounds:

In [ ]:
x = coordinates(N); fx = N*coordinates(N)
n = ifft(pad_mid(pswf, N))
pylab.semilogy(coordinates(4*int(xN_size))*4*xN_size/N,
               extract_mid(numpy.abs(ifft(pad_mid(pswf, N))),4*int(xN_size)));
pylab.legend(["n"]);
mark_range("$x_n$", -xN,xN); pylab.xlim(-2*int(xN_size)/N, (2*int(xN_size)-1)/N); pylab.show();
pylab.semilogy(coordinates(yN_size)*yN_size, pswf); pylab.legend(["$\\mathcal{F}[n]$"]);
mark_range("$y_B$", -yB,yB); pylab.xlim(-N//2,N//2-1); mark_range("$y_n$", -yN,yN); pylab.show();

In [ ]:
FAG = fft(pad_mid(numpy.ones(xA_size), N))
@interact(l0= (-1,1,0.01), phi = (0, 2, 0.01), w_phi2 = (0,1000, 1))
def test_noncoplanar(l0, phi, w_phi2):
    w = w_phi2 / phi**2
    print("w=", w)
    l0 = -phi * (N-yN_size) / 4 / N
    print('l0=', l0)
    FG = numpy.exp(2.j * numpy.pi * w * numpy.sqrt(1 - (l0 + coordinates(N) * phi)**2))
    Fn = pad_mid(pswf, N)
    FnG = Fn * FG
    Gn = numpy.abs(ifft(FG * Fn))
    pylab.semilogy(coordinates(N), numpy.abs(ifft(FG)))
    pylab.semilogy(coordinates(N), numpy.abs(ifft(Fn)))
    pylab.semilogy(coordinates(N), numpy.abs(Gn))
    pylab.semilogy(coordinates(N), numpy.abs(ifft(Fn*FAG)))
    pylab.semilogy(coordinates(N), numpy.abs(ifft(FnG*FAG)))
    pylab.legend(['G', 'n', 'G*n', 'n*A', 'G*n*A'])
    mark_range('W', -W/2/res,W/2/res)
    mark_range('W', -W/2/res-1/4,W/2/res+1/4)
    p0 = numpy.searchsorted(Gn[:res//2], err_base[W])
    print(Gn[p0-1], err_base[W], Gn[p0])
    p0 = (p0 * (err_base[W] - Gn[p0-1]) +
          (p0 - 1) * (Gn[p0] - err_base[W])) / (Gn[p0]-Gn[p0-1])
    p = res//2 - p0
    #mark_range('p', -p/res,p/res)
    #mark_range('p', -p/res-1/4,p/res+1/4)
    print(p-W/2)
    pylab.grid()

In [ ]:
from ipywidgets import interact_manual

@interact_manual
def export_pswf(pswf_path = "../../data/grid/T06_pswf_%s.in" % cfg_name):
    with open(pswf_path, "w") as f:
        numpy.fft.ifftshift(pswf).tofile(f)
    print("wrote %s" % pswf_path)

Calculate actual work terms to use. We need both $n$ and $b$ in image space.

In [ ]:
Fb = 1/extract_mid(pswf, yB_size)
Fn = pswf[(yN_size//2)%int(1/2/xM)::int(1/2/xM)]
facet_m0_trunc = pswf * numpy.sinc(coordinates(yN_size)*xM_size/N*yN_size)
facet_m0_trunc = xM_size*yP_size/N * extract_mid(ifft(pad_mid(facet_m0_trunc, yP_size)), xMxN_yP_size).real
pylab.semilogy(coordinates(xMxN_yP_size)/yP_size*xMxN_yP_size, facet_m0_trunc); mark_range("xM", -xM, xM)

In [ ]:
FAG = fft(pad_mid(numpy.ones(xA_size), N))
@interact(max_l0= (0,1,0.01), phi = (0, 2, 0.01), w = (0,1000, 1))
def test_noncoplanar(max_l0, phi, w):
    FG = numpy.exp(2.j * numpy.pi * w * numpy.sqrt(1 - (max_l0 + coordinates(yB_size) * phi)**2))
    Fn = pad_mid(pswf, N)
    FnG = Fn * FG
    Gn = numpy.abs(ifft(FG * Fn))
    pylab.semilogy(coordinates(N), numpy.abs(ifft(FG)))
    pylab.semilogy(coordinates(N), numpy.abs(ifft(Fn)))
    pylab.semilogy(coordinates(N), numpy.abs(Gn))
    pylab.semilogy(coordinates(N), numpy.abs(ifft(Fn*FAG)))
    pylab.semilogy(coordinates(N), numpy.abs(ifft(FnG*FAG)))
    pylab.legend(['G', 'n', 'G*n', 'n*A', 'G*n*A'])
    mark_range('W', -W/2/res,W/2/res)
    mark_range('W', -W/2/res-1/4,W/2/res+1/4)
    p0 = numpy.searchsorted(Gn[:res//2], err_base[W])
    print(Gn[p0-1], err_base[W], Gn[p0])
    p0 = (p0 * (err_base[W] - Gn[p0-1]) +
          (p0 - 1) * (Gn[p0] - err_base[W])) / (Gn[p0]-Gn[p0-1])
    p = res//2 - p0
    #mark_range('p', -p/res,p/res)
    #mark_range('p', -p/res-1/4,p/res+1/4)
    print(p-W/2)
    pylab.grid()

## Layout subgrids + facets

In [ ]:
nsubgrid = int(math.ceil(N / xA_size))
nfacet = int(math.ceil(N / yB_size))
print("%d subgrids, %d facets needed to cover" % (nsubgrid, nfacet))
subgrid_off = xA_size * numpy.arange(nsubgrid)
facet_off = yB_size * numpy.arange(nfacet)
def whole(xs): return numpy.all(numpy.abs(xs - numpy.around(xs)) < 1e-13)
assert whole(numpy.outer(subgrid_off, facet_off) / N)
assert whole(facet_off*xM_size/N)

We need a bunch of array constants derived from the gridding function:
 * $\mathcal Fb$ ($y_B$ size)
 * $\mathcal Fn$ ($y_N$ size, sampled at $x_M$ rate), as well as 
 * $\mathcal Fm' = \mathcal Fn\mathcal Fm$ term ($y_P$ size, sampled at $x_M+x_N$).
 
For the convolution with $b$, $n$, and cheap multiplication with $m$ at $y_P$ image size respectively.

Determine subgrid/facet offsets and the appropriate A/B masks for cutting them out. We are aiming for full coverage here: Every pixel is part of exactly one subgrid / facet.

In [ ]:
subgrid_A = numpy.zeros((nsubgrid, xA_size), dtype=int)
subgrid_border = (subgrid_off + numpy.hstack([subgrid_off[1:],[N]])) // 2
for i in range(nsubgrid):
    left = (subgrid_border[i-1] - subgrid_off[i] + xA_size//2) % N
    right = subgrid_border[i] - subgrid_off[i] + xA_size//2
    assert left >= 0 and right <= xA_size, "xA not large enough to cover subgrids!"
    subgrid_A[i,left:right] = 1

facet_B = numpy.zeros((nfacet, yB_size), dtype=bool)
facet_split = numpy.array_split(range(N), nfacet)
facet_border = (facet_off + numpy.hstack([facet_off[1:],[N]])) // 2
for j in range(nfacet):
    left = (facet_border[j-1] - facet_off[j] + yB_size//2) % N
    right = facet_border[j] - facet_off[j] + yB_size//2
    assert left >= 0 and right <= yB_size, "yB not large enough to cover facets!"
    facet_B[j,left:right] = 1

Now calculate the actual subgrids & facets

In [ ]:
def make_subgrid_and_facet(G):
    FG = fft(G)
    subgrid = numpy.empty((nsubgrid, xA_size), dtype=complex)
    for i in range(nsubgrid):
        subgrid[i] = subgrid_A[i] * extract_mid(numpy.roll(G, -subgrid_off[i]), xA_size)
    facet = numpy.empty((nfacet, yB_size), dtype=complex)
    for j in range(nfacet):
        facet[j] = facet_B[j] * extract_mid(numpy.roll(FG, -facet_off[j]), yB_size)
    return subgrid, facet
subgrid, facet = make_subgrid_and_facet(numpy.random.rand(N)-0.5)

## Facet $\rightarrow$ Subgrid

With a few more slight optimisations we arrive at a compact representation for our algorithm:

In [ ]:
xN_yP_size = xMxN_yP_size - xM_yP_size
def facets_to_subgrid_1(facet):
    RNjMiBjFj = numpy.empty((nsubgrid, nfacet, xM_yN_size), dtype=complex)
    for j in range(nfacet):
        BjFj = ifft(pad_mid(facet[j] * Fb, yP_size))
        for i in range(nsubgrid):
            MiBjFj = facet_m0_trunc * extract_mid(numpy.roll(BjFj, -subgrid_off[i]*yP_size//N), xMxN_yP_size)
            MiBjFj_sum = numpy.array(extract_mid(MiBjFj, xM_yP_size))
            MiBjFj_sum[:xN_yP_size//2] += MiBjFj[-xN_yP_size//2:]
            MiBjFj_sum[-xN_yP_size//2:] += MiBjFj[:xN_yP_size//2:]
            RNjMiBjFj[i,j] = Fn * extract_mid(fft(MiBjFj_sum), xM_yN_size)
    return RNjMiBjFj
def facets_to_subgrid_2(nmbfs, i):
    approx = numpy.zeros(xM_size, dtype=complex)
    for j in range(nfacet):
        approx += numpy.roll(pad_mid(nmbfs[i,j], xM_size), facet_off[j]*xM_size//N)
    return subgrid_A[i] * extract_mid(ifft(approx), xA_size)

print("Facet data:", facet.shape, facet.size)
nmbfs = facets_to_subgrid_1(facet)
# - redistribution of nmbfs here -
print("Redistributed data:", nmbfs.shape, nmbfs.size, " overhead:", nmbfs.size / facet.size)
approx_subgrid = [ facets_to_subgrid_2(nmbfs, i) for i in range(nsubgrid) ]

Let us look at the error terms:

In [ ]:
fig = pylab.figure(figsize=(16, 8))
ax1, ax2 = fig.add_subplot(211), fig.add_subplot(212)
err_sum = err_sum_img = 0
for i in range(nsubgrid):
    error = approx_subgrid[i] - subgrid[i]
    ax1.semilogy(xA*2*coordinates(xA_size), numpy.abs(error))
    ax2.semilogy(N*coordinates(xA_size), numpy.abs(fft(error)))
    err_sum += numpy.abs(error)**2 / nsubgrid
    err_sum_img += numpy.abs(fft(error))**2 / nsubgrid
mark_range("$x_A$", -xA, xA, ax=ax1); mark_range("$N/2$", -N/2, N/2, ax=ax2)
print("RMSE:", numpy.sqrt(numpy.mean(err_sum)), "(image:", numpy.sqrt(numpy.mean(err_sum_img)), ")")

By feeding the implementation single-pixel inputs we can create a full error map:

In [ ]:
if N <= 1024:
    error_map = []
    for xs in range(-N//2, N//2):
        if xs % 128 == 0:
            print(xs, end=' ')
        FG = numpy.zeros(N); FG[xs + N//2] = 1
        subgrid, facet = make_subgrid_and_facet(ifft(FG))
        nmbfs = facets_to_subgrid_1(facet)
        err_map_row = numpy.zeros(N, dtype=complex)
        for i in range(nsubgrid):
            error = facets_to_subgrid_2(nmbfs, i) - subgrid[i]
            err_map_row += numpy.roll(pad_mid(error, N), subgrid_off[i])
        error_map.append(fft(err_map_row))

In [ ]:
if N <= 1024:
    err_abs = numpy.abs(error_map)
    # Filter out spurious zeroes that would cause division-by-zero
    err_log = numpy.log(numpy.maximum(numpy.min(err_abs[err_abs>0]), err_abs))/numpy.log(10)
    pylab.figure(figsize=(20,20))
    pylab.imshow(err_log, cmap=pylab.get_cmap('inferno'), norm=colors.PowerNorm(gamma=2.0),
                 extent=(-N//2,N//2,-N//2,N//2));
    pylab.colorbar(shrink=0.6); pylab.ylabel('in'); pylab.xlabel('out');
    pylab.title('Output error depending on input pixel (absolute log10)');

In [ ]:
if N <= 1024:
    worst_rmse = 0; worst_err = 0
    for xs in range(N):
        rmse = numpy.sqrt(numpy.mean(numpy.abs(error_map[xs])**2))
        if rmse > worst_rmse: worst_rmse = rmse; worst_err = error_map[xs]
    pylab.semilogy(numpy.abs(worst_err))

In [ ]:
if N <= 1024:
    worst_rmse = 0; worst_err = 0; em = numpy.array(error_map)
    for xs in range(N):
        rmse = numpy.sqrt(numpy.mean(numpy.abs(em[:,xs])**2))
        if rmse > worst_rmse: worst_rmse = rmse; worst_err = em[:,xs]
    pylab.semilogy(numpy.abs(worst_err))

## Subgrid $\rightarrow$ facet

The other direction works similarly, now we want:
$$F_j \approx b_j \ast \sum_i m_i (n_j \ast S_i)$$

We run into a very similar problem with $m$ as when reconstructing subgrids, except this time it happens because we want to construct:
$$ b_j \left( m_i (n_j \ast S_i)\right)
  = b_j \left( \mathcal F^{-1}\left[\Pi_{2y_P} \mathcal F m_i\right] (n_j \ast S_i)\right)$$

As usual, this is entirely dual: In the previous case we had a signal limited by $y_B$ and needed the result of the convolution up to $y_N$, whereas now we have a signal bounded by $y_N$, but need the convolution result up to $y_B$. This cancels out - therefore we are okay with the same choice of $y_P$.

In [ ]:
def subgrid_to_facet_1(subgrid):
    FNjSi = numpy.empty((nsubgrid, nfacet, xM_yN_size), dtype=complex)
    for i in range(nsubgrid):
        FSi = fft(pad_mid(subgrid[i], xM_size))
        for j in range(nfacet):
            FNjSi[i,j] = extract_mid(numpy.roll(FSi, -facet_off[j]*xM_size//N), xM_yN_size)
    return Fn * FNjSi

def subgrid_to_facet_2(nafs, j):
    approx = numpy.zeros(yB_size, dtype=complex)
    for i in range(nsubgrid):
        NjSi = numpy.zeros(xMxN_yP_size, dtype=complex)
        NjSi_mid = extract_mid(NjSi, xM_yP_size)
        NjSi_mid[:] = ifft(pad_mid(nafs[i,j], xM_yP_size)) # updates NjSi via reference!
        NjSi[-xN_yP_size//2:] = NjSi_mid[:xN_yP_size//2]
        NjSi[:xN_yP_size//2:] = NjSi_mid[-xN_yP_size//2:]
        FMiNjSi = fft(numpy.roll(pad_mid(facet_m0_trunc * NjSi, yP_size), subgrid_off[i]*yP_size//N))
        approx += extract_mid(FMiNjSi, yB_size)
    return approx * Fb * facet_B[j]

print("Subgrid data:", subgrid.shape, subgrid.size)
nafs = subgrid_to_facet_1(subgrid)

# - redistribution of FNjSi here -
print("Intermediate data:", nafs.shape, nafs.size)
approx_facet = [ subgrid_to_facet_2(nafs, j) for j in range(nfacet) ]

In [ ]:
fig = pylab.figure(figsize=(16, 8))
ax1, ax2 = fig.add_subplot(211), fig.add_subplot(212)
err_sum = err_sum_img = 0
for j in range(nfacet):
    error = approx_facet[j] - facet[j]
    err_sum += numpy.abs(ifft(error))**2
    err_sum_img += numpy.abs(error)**2
    ax1.semilogy(coordinates(yB_size), numpy.abs(ifft(error)))
    ax2.semilogy(yB_size*coordinates(yB_size), numpy.abs(error))
print("RMSE:", numpy.sqrt(numpy.mean(err_sum)), "(image:", numpy.sqrt(numpy.mean(err_sum_img)), ")")
mark_range("$x_A$", -xA, xA, ax=ax1); mark_range("$x_M$", -xM, xM, ax=ax1)
mark_range("$y_B$", -yB, yB, ax=ax2); mark_range("$0.5$", -.5, .5, ax=ax1)
pylab.show(fig)

In [ ]:
error_map_2 = []
for xs in range(-N//2, N//2):
    if xs % 128 == 0:
        print(xs, end=' ')
    FG = numpy.zeros(N); FG[xs + N//2] = 1
    subgrid, facet = make_subgrid_and_facet(ifft(FG))
    nafs = subgrid_to_facet_1(subgrid)

    err_sum_hq = numpy.zeros(N, dtype=complex)
    
    for j in range(nfacet):
        approx  = subgrid_to_facet_2(nafs, j)
        err_sum_hq += numpy.roll(pad_mid(approx - facet[j], N), facet_off[j])
    error_map_2.append(err_sum_hq)

In [ ]:
err_abs = numpy.abs(error_map_2)
# Filter out spurious zeroes that would cause division-by-zero
err_log = numpy.log(numpy.maximum(numpy.min(err_abs[err_abs>0]), err_abs))/numpy.log(10)
pylab.figure(figsize=(20,20))
pylab.imshow(err_log, cmap=pylab.get_cmap('inferno'), norm=colors.PowerNorm(gamma=2.0),
             extent=(-N//2,N//2,-N//2,N//2));
pylab.colorbar(shrink=0.6); pylab.ylabel('in'); pylab.xlabel('out');
pylab.title('Output error depending on input pixel (absolute log10)');
tikzplotlib.save('error_map_2.tikz', axis_height='3.5cm', axis_width='\\textwidth', textsize=5, show_info=False);

In [ ]:
worst_rmse = 0; worst_err = 0
for xs in range(N):
    rmse = numpy.sqrt(numpy.mean(numpy.abs(error_map_2[xs])**2))
    if rmse > worst_rmse: worst_rmse = rmse; worst_err = error_map[xs]
pylab.semilogy(numpy.abs(worst_err))

## 2D case

All of this generalises to two dimensions in the way you would expect. Let us set up test data:

In [ ]:
print(nsubgrid,"x",nsubgrid,"subgrids,",nfacet,"x", nfacet,"facets")
subgrid_2 = numpy.empty((nsubgrid, nsubgrid, xA_size, xA_size), dtype=complex)
facet_2 = numpy.empty((nfacet, nfacet, yB_size, yB_size), dtype=complex)

G_2 = numpy.exp(2j*numpy.pi*numpy.random.rand(N,N))*numpy.random.rand(N,N)/2
FG_2 = fft(G_2)

FG_2 = numpy.zeros((N,N))
source_count = 1000
sources = [ (numpy.random.randint(-N//2, N//2-1), numpy.random.randint(-N//2, N//2-1),
             numpy.random.rand() * N * N / numpy.sqrt(source_count) / 2) for _ in range(source_count) ]
for x, y, i in sources:
    FG_2[y+N//2,x+N//2] += i
G_2 = ifft(FG_2)
print("Mean grid absolute:", numpy.mean(numpy.abs(G_2)))

for i0,i1 in itertools.product(range(nsubgrid), range(nsubgrid)):
    subgrid_2[i0,i1] = extract_mid(numpy.roll(G_2, (-subgrid_off[i0], -subgrid_off[i1]), (0,1)), xA_size)
    subgrid_2[i0,i1] *= numpy.outer(subgrid_A[i0], subgrid_A[i1])
for j0,j1 in itertools.product(range(nfacet), range(nfacet)):
    facet_2[j0,j1] = extract_mid(numpy.roll(FG_2, (-facet_off[j0], -facet_off[j1]), (0,1)), yB_size)
    facet_2[j0,j1] *= numpy.outer(facet_B[j0], facet_B[j1])

Given that the amount of data has been squared, performance is a bit more of a concern now. Fortunately, the entire procedure is completely separable, so let us first re-define the operations to work on one array axis exclusively:

In [ ]:
def slice_a(fill_val, axis_val, dims, axis):
    return tuple([ axis_val if i == axis else fill_val for i in range(dims) ])
def pad_mid_a(a, N, axis):
    N0 = a.shape[axis]
    if N == N0: return a
    pad = slice_a((0,0), (N//2-N0//2, (N+1)//2-(N0+1)//2), 
                  len(a.shape), axis)    
    return numpy.pad(a, pad, mode='constant', constant_values=0.0)
def extract_mid_a(a, N, axis):
    assert N <= a.shape[axis]
    cx = a.shape[axis] // 2
    if N % 2 != 0:
        slc = slice(cx - N // 2, cx + N // 2 + 1)
    else:
        slc = slice(cx - N // 2, cx + N // 2)
    return a[slice_a(slice(None), slc, len(a.shape), axis)]
def fft_a(a, axis):
    return numpy.fft.fftshift(numpy.fft.fft(numpy.fft.ifftshift(a, axis),axis=axis),axis)
def ifft_a(a, axis):
    return numpy.fft.fftshift(numpy.fft.ifft(numpy.fft.ifftshift(a, axis),axis=axis),axis)
def broadcast_a(a, dims, axis):
    slc = [numpy.newaxis] * dims
    slc[axis] = slice(None)
    return a[slc]
def broadcast_a(a, dims, axis):
    return a[slice_a(numpy.newaxis, slice(None), dims, axis)]

This allows us to define the two fundamental operations - going from $F$ to $b\ast F$ and from $b\ast F$ to $n\ast m(b\ast F)$ separately:

In [ ]:
def prepare_facet(facet, axis):
    BF = pad_mid_a(facet * broadcast_a(Fb, len(facet.shape), axis), yP_size, axis)
    BF = ifft_a(BF, axis)
    return BF
def extract_subgrid(BF, i, axis):
    dims = len(BF.shape)
    BF_mid = extract_mid_a(numpy.roll(BF, -subgrid_off[i]*yP_size//N, axis), xMxN_yP_size, axis)
    MBF = broadcast_a(facet_m0_trunc,dims,axis) * BF_mid
    MBF_sum = numpy.array(extract_mid_a(MBF, xM_yP_size, axis))
    xN_yP_size = xMxN_yP_size - xM_yP_size
    # [:xN_yP_size//2] / [-xN_yP_size//2:] for axis, [:] otherwise
    slc1 = slice_a(slice(None), slice(xN_yP_size//2), dims, axis)
    slc2 = slice_a(slice(None), slice(-xN_yP_size//2,None), dims, axis)
    MBF_sum[slc1] += MBF[slc2]
    MBF_sum[slc2] += MBF[slc1]
    return broadcast_a(Fn,len(BF.shape),axis) * \
           extract_mid_a(fft_a(MBF_sum, axis), xM_yN_size, axis)

Having those operations separately means that we can shuffle things around quite a bit without affecting the result. The obvious first choice might be to do all facet-preparation up-front, as this allows us to share the computation across all subgrids:

In [ ]:
t = time.time()
NMBF_NMBF = numpy.empty((nsubgrid, nsubgrid, nfacet, nfacet, xM_yN_size, xM_yN_size), dtype=complex)
for j0,j1 in itertools.product(range(nfacet), range(nfacet)):
    BF_F = prepare_facet(facet_2[j0,j1], 0)
    BF_BF = prepare_facet(BF_F, 1)
    for i0 in range(nsubgrid):
        NMBF_BF = extract_subgrid(BF_BF, i0, 0)
        for i1 in range(nsubgrid):
            NMBF_NMBF[i0,i1,j0,j1] = extract_subgrid(NMBF_BF, i1, 1)
print(time.time() - t, "s")

However, remember that `prepare_facet` increases the amount of data involved, which in turn means that we need to shuffle more data through subsequent computations.

Therefore it is actually more efficient to first do the subgrid-specific reduction, and *then* continue with the (constant) facet preparation along the other axis. We can tackle both axes in whatever order we like, it doesn't make a difference for the result:

In [ ]:
t = time.time()
for j0,j1 in itertools.product(range(nfacet), range(nfacet)):
    BF_F = prepare_facet(facet_2[j0,j1], 0)
    for i0 in range(nsubgrid):
        NMBF_F = extract_subgrid(BF_F, i0, 0)
        NMBF_BF = prepare_facet(NMBF_F, 1)
        for i1 in range(nsubgrid):
            NMBF_NMBF[i0,i1,j0,j1] = extract_subgrid(NMBF_BF, i1, 1)
print(time.time() - t, "s")

In [ ]:
t = time.time()
for j0,j1 in itertools.product(range(nfacet), range(nfacet)):
    F_BF = prepare_facet(facet_2[j0,j1], 1)
    for i1 in range(nsubgrid):
        F_NMBF = extract_subgrid(F_BF, i1, 1)
        BF_NMBF = prepare_facet(F_NMBF, 0)
        for i0 in range(nsubgrid):
            NMBF_NMBF[i0,i1,j0,j1] = extract_subgrid(BF_NMBF, i0, 0)
print(time.time() - t, "s")

In [ ]:
pylab.rcParams['figure.figsize'] = 16, 8
err_mean = err_mean_img = 0
for i0,i1 in itertools.product(range(nsubgrid), range(nsubgrid)):
    approx = numpy.zeros((xM_size, xM_size), dtype=complex)
    for j0,j1 in itertools.product(range(nfacet), range(nfacet)):
        approx += numpy.roll(pad_mid(NMBF_NMBF[i0,i1,j0,j1], xM_size),
                             (facet_off[j0]*xM_size//N, facet_off[j1]*xM_size//N), (0,1))
    approx = extract_mid(ifft(approx), xA_size)
    approx *= numpy.outer(subgrid_A[i0], subgrid_A[i1])
    err_mean += numpy.abs(approx - subgrid_2[i0,i1])**2 / nsubgrid**2
    err_mean_img += numpy.abs(fft(approx - subgrid_2[i0,i1]))**2 / nsubgrid**2
pylab.imshow(numpy.log(numpy.sqrt(err_mean)) / numpy.log(10)); pylab.colorbar(); pylab.show()
pylab.imshow(numpy.log(numpy.sqrt(err_mean_img)) / numpy.log(10)); pylab.colorbar(); pylab.show()
print("RMSE:", numpy.sqrt(numpy.mean(err_mean)), "(image:", numpy.sqrt(numpy.mean(err_mean_img)), ")")

In [ ]:
@interact(xs=(0,N), ys=(0,N))
def test_accuracy(xs=252,ys=252):
    subgrid_2 = numpy.empty((nsubgrid, nsubgrid, xA_size, xA_size), dtype=complex)
    facet_2 = numpy.empty((nfacet, nfacet, yB_size, yB_size), dtype=complex)

    #G_2 = numpy.exp(2j*numpy.pi*numpy.random.rand(N,N))*numpy.random.rand(N,N)/2
    #FG_2 = fft(G_2)

    FG_2 = numpy.zeros((N,N))
    FG_2[ys,xs] = 1
    G_2 = ifft(FG_2)

    for i0,i1 in itertools.product(range(nsubgrid), range(nsubgrid)):
        subgrid_2[i0,i1] = extract_mid(numpy.roll(G_2, (-subgrid_off[i0], -subgrid_off[i1]), (0,1)), xA_size)
        subgrid_2[i0,i1] *= numpy.outer(subgrid_A[i0], subgrid_A[i1])
    for j0,j1 in itertools.product(range(nfacet), range(nfacet)):
        facet_2[j0,j1] = extract_mid(numpy.roll(FG_2, (-facet_off[j0], -facet_off[j1]), (0,1)), yB_size)
        facet_2[j0,j1] *= numpy.outer(facet_B[j0], facet_B[j1])

    NMBF_NMBF = numpy.empty((nsubgrid, nsubgrid, nfacet, nfacet, xM_yN_size, xM_yN_size), dtype=complex)
    for j0,j1 in itertools.product(range(nfacet), range(nfacet)):
        BF_F = prepare_facet(facet_2[j0,j1], 0)
        BF_BF = prepare_facet(BF_F, 1)
        for i0 in range(nsubgrid):
            NMBF_BF = extract_subgrid(BF_BF, i0, 0)
            for i1 in range(nsubgrid):
                NMBF_NMBF[i0,i1,j0,j1] = extract_subgrid(NMBF_BF, i1, 1)

    pylab.rcParams['figure.figsize'] = 16, 8
    err_mean = err_mean_img = 0
    for i0,i1 in itertools.product(range(nsubgrid), range(nsubgrid)):
        approx = numpy.zeros((xM_size, xM_size), dtype=complex)
        for j0,j1 in itertools.product(range(nfacet), range(nfacet)):
            approx += numpy.roll(pad_mid(NMBF_NMBF[i0,i1,j0,j1], xM_size),
                                 (facet_off[j0]*xM_size//N, facet_off[j1]*xM_size//N), (0,1))
        approx = extract_mid(ifft(approx), xA_size)
        approx *= numpy.outer(subgrid_A[i0], subgrid_A[i1])
        err_mean += numpy.abs(approx - subgrid_2[i0,i1])**2 / nsubgrid**2
        err_mean_img += numpy.abs(fft(approx - subgrid_2[i0,i1]))**2 / nsubgrid**2
    #pylab.imshow(numpy.log(numpy.sqrt(err_mean)) / numpy.log(10)); pylab.colorbar(); pylab.show()
    #pylab.imshow(numpy.log(numpy.sqrt(err_mean_img)) / numpy.log(10)); pylab.colorbar(); pylab.show()
    print("RMSE:", numpy.sqrt(numpy.mean(err_mean)), "(image:", numpy.sqrt(numpy.mean(err_mean_img)), ")")

252/252: RMSE: 9.124879530036322e-16 (image: 1.751976869766974e-13 )
252/1310: RMSE: 2.9755045512714923e-12 (image: 5.712968738441266e-10 )
1307/1310: RMSE: 4.349813048259021e-12 (image: 8.35164105265732e-10 )
1308/1310: RMSE: 4.726101167659007e-12 (image: 9.074114241905292e-10 )
1309/1310: RMSE: 4.693531621846021e-12 (image: 9.011580713944359e-10 )
1310/1310: RMSE: 4.208376099939205e-12 (image: 8.080082111883273e-10 )
1311/1310: RMSE: 3.4437254455821605e-12 (image: 6.611952855517747e-10 )
1308/1308: RMSE: 5.192955275658332e-12 (image: 9.970474129263996e-10 )

## Degridding

To use this for radio astronomy, our goal in this context is to (de)grid visibilities from subgrids. This uses very similar machinery - in fact, what we described so far can simply be re-expressed as gridding or degridding all points of a sub-grid using facets. Difference being that our method is a lot faster and requires less data movement.

However, this does similarity does not actually buy us much: While for the recombination we consider the fields of view of facets, for gridding visibilities we are interested in the "global" field of view. Therefore we need a different grid correction and gridder that gets applied before and after we have done the combination, respectively.

The full size of the considered image is fixed to $N$, therefore our effective image size is $2x_0N$:

In [ ]:
pylab.rcParams['figure.figsize'] = 16, 4

gc_alpha = 0; xGp = 4/N; gc_x0 = 0.3 # 0.125
gc_support = int(2*xGp*N)
print("parameter:", numpy.pi*gc_support/2, 'support:', gc_support, "x0:", gc_x0)
x0_size = int(N*gc_x0*2)
gc_pswf = anti_aliasing_function(N, gc_alpha, numpy.pi*gc_support/2)
gc = pad_mid(extract_mid(1 / gc_pswf, x0_size), N)
pylab.semilogy(x0_size*coordinates(x0_size), numpy.abs(extract_mid(gc, x0_size))); pylab.legend(["F[n]"]);
pylab.xlim((-N/1.8, N/1.8))
mark_range("$x_0N$", -gc_x0*N,gc_x0*N);
mark_range("$N/2$", -N/2,N/2); pylab.title("Grid correction"); pylab.show();

From this we derive the new $\mathcal F G$ that we are going to feed to the recombination algorithm:

In [ ]:
cropped_sources = [ (l,m,i) for l,m,i in sources
                    if l >= -x0_size / 2 and l < x0_size / 2 and \
                       m >= -x0_size / 2 and m < x0_size / 2 ]
print(f"{len(cropped_sources)} / {len(sources)} sources still in view after cropping")
FG_2_gc = FG_2 * numpy.outer(gc, gc)
G_2_gc = ifft(FG_2_gc)
crop = pad_mid(numpy.ones(x0_size), N)
G_2_cropped = ifft(FG_2 * numpy.outer(crop,crop))
show_image(numpy.log(numpy.maximum(1e-15, numpy.abs(fft(G_2_cropped)))) / numpy.log(10), "FG_2_cropped", N)

Test base performance of gridder:

In [ ]:
def dft(sources, theta, u,v,w):
    actual = 0
    for l, m, i in sources:
        n = 1 - numpy.sqrt(1 - (l/N*theta)**2 - (m/N*theta)**2)
        actual += i * numpy.exp(2j * numpy.pi * (u * l + v * m + w * n))
    return actual / N / N

@interact(iu=(0, N, 0.1),iv=(0, N, 0.1), theta=(0,2,0.01), w=(-2000,2000,0.5))
def test_degrid_accuracy(iu,iv=N/2,w=1000, theta=0.3):
    u = (iu - N//2) / N; v = (iv - N//2) / N
    actual = dft(cropped_sources,theta,u,v,w)
    G_2_gc_= G_2_gc
    if w != 0:
        l,m = numpy.meshgrid(theta*coordinates(N), theta*coordinates(N))
        n = 1 - numpy.sqrt(1 - l**2 - m**2)
        G_2_gc_ = ifft(fft(G_2_gc_) * numpy.exp(2j * numpy.pi * w * n))
    deg = conv_predict(N, 1, numpy.array([(u,v,0)]), None, G_2_gc_, kernel)[0]
    print("actual:       ", actual)
    print("degridded:    ", deg)
    print("degrid error: ", numpy.abs(deg-actual))

Which in turn leads to new facets. Note how the grid correction pattern is clearly larger than any individual facet.

The other thing to notice here is that due to the grid correction margin a significant portion of the image is now zero, which translates to entire facets being zero. Due to the linearity of the method this means we could simply skip those. For the purpose of this notebook we do not use this, but it is a good optimisation to keep in mind.

In [ ]:
subgrid_2 = numpy.empty((nsubgrid, nsubgrid, xA_size, xA_size), dtype=complex)
facet_2 = numpy.empty((nfacet, nfacet, yB_size, yB_size), dtype=complex)
for i0,i1 in itertools.product(range(nsubgrid), range(nsubgrid)):
    subgrid_2[i0,i1] = extract_mid(numpy.roll(G_2_gc, (-subgrid_off[i0], -subgrid_off[i1]), (0,1)), xA_size)
    subgrid_2[i0,i1] *= numpy.outer(subgrid_A[i0], subgrid_A[i1])
fig = pylab.figure(figsize=(32,32))
for j0,j1 in itertools.product(range(nfacet), range(nfacet)):
    facet_2[j0,j1] = extract_mid(numpy.roll(FG_2_gc, (-facet_off[j0], -facet_off[j1]), (0,1)), yB_size)
    facet_2[j0,j1] *= numpy.outer(facet_B[j0], facet_B[j1])
    show_image(numpy.log(numpy.maximum(1e-15, numpy.abs(facet_2[j0,j1]))) / numpy.log(10),
               "facet_%d%d" % (j0,j1), N, axes=fig.add_subplot(nfacet,nfacet,j1+(nfacet-j0-1)*nfacet+1),
              norm=(-15,8))
pylab.show(fig)

The recombination algorithm again, using the new data.

In [ ]:
NMBF_NMBF = numpy.empty((nsubgrid, nsubgrid, nfacet, nfacet, xM_yN_size, xM_yN_size), dtype=complex)
for j0,j1 in itertools.product(range(nfacet), range(nfacet)):
    F_BF = prepare_facet(facet_2[j0,j1], 1)
    for i1 in range(nsubgrid):
        F_NMBF = extract_subgrid(F_BF, i1, 1)
        BF_NMBF = prepare_facet(F_NMBF, 0)
        for i0 in range(nsubgrid):
            NMBF_NMBF[i0,i1,j0,j1] = extract_subgrid(BF_NMBF, i0, 0)

from pylru import lrudecorator
@lrudecorator(100)
def make_approx_subgrid(i0,i1):
    approx = numpy.zeros((xM_size, xM_size), dtype=complex)
    for j0,j1 in itertools.product(range(nfacet), range(nfacet)):
        approx += numpy.roll(pad_mid(NMBF_NMBF[i0,i1,j0,j1], xM_size),
                             (facet_off[j0]*xM_size//N, facet_off[j1]*xM_size//N), (0,1))
    # Extract region that is set in subgrid for comparison
    approx_compare = extract_mid(ifft(approx), xA_size)
    approx_compare *= numpy.outer(subgrid_A[i0], subgrid_A[i1])
    rmse = numpy.sqrt(numpy.mean(numpy.abs(approx_compare - subgrid_2[i0,i1])**2 / nsubgrid**2))
    # Return full approximation. We degrid from it, so bounds don't matter
    return ifft(approx), rmse / numpy.mean(numpy.abs(approx_compare))

In order to obtain visibilities at non-integer positions we need an oversampled gridding function, as usual:

In [ ]:
oversample = 2**14
print("grid support:", gc_support)
print("oversampling:", oversample)
kernel = kernel_oversample(gc_pswf, oversample, gc_support).real
kernel /= numpy.sum(kernel[0])
r = numpy.arange(-oversample*(gc_support//2), oversample*((gc_support+1)//2)) / oversample
pylab.semilogy(r, numpy.transpose(kernel).flatten().real); mark_range("$Nx_G$", -N*xGp,N*xGp);
pylab.title("Gridding kernel (oversampled x%d)" % oversample); pylab.show();

In [ ]:
@interact(iu=(0, N, 0.1),iv=(0, N, 0.1), theta=(0,2,0.01), w=(-2000,2000,0.5))
def test_degrid_accuracy(iu,iv=N/2,w=1000, theta=0.3, show_subgrid=True):
    u = (iu - N//2) / N; v = (iv - N//2) / N
    su = numpy.sum((iu+N//2)%N >= subgrid_border) % nsubgrid
    sv = numpy.sum((iv+N//2)%N >= subgrid_border) % nsubgrid
    siu = iu + xA_size//2-(subgrid_off[su] + N//2) % N
    siv = iv + xA_size//2-(subgrid_off[sv] + N//2) % N
    
    # Predict from grid
    G_2_gc_= G_2_gc
    if w != 0:
        l,m = numpy.meshgrid(theta*coordinates(N), theta*coordinates(N))
        n = 1 - numpy.sqrt(1 - l**2 - m**2)
        G_2_gc_ = ifft(fft(G_2_gc_) * numpy.exp(2j * numpy.pi * w * n))
    deg = conv_predict(N, 1, numpy.array([(u,v,0)]), None, G_2_gc_, kernel)[0]
    actual = dft(cropped_sources, theta, u,v,w)
    print("actual:       ", actual)
    print("degridded:    ", deg)
    print("degrid error: ", numpy.abs(deg-actual))

    true_subgrid =  extract_mid(numpy.roll(G_2_gc_, (-subgrid_off[sv], -subgrid_off[su]), (0,1)), xM_size)

    approx_subgrid, rmse = make_approx_subgrid(sv, su)
    if w != 0:
        l,m = numpy.meshgrid(theta*coordinates(xM_size), theta*coordinates(xM_size))
        n = 1 - numpy.sqrt(1 - l**2 - m**2)
        approx_subgrid = ifft(fft(approx_subgrid) * numpy.exp(2j * numpy.pi * w * n))
    print("subgrid:       (%d/%d), rmse: %g" % (su, sv, rmse))
    
    sou = (((subgrid_off[su] + N//2) % N) - N//2) / N
    sov = (((subgrid_off[sv] + N//2) % N) - N//2) / N
    deg_ap = conv_predict(N, 2*xM, numpy.array([(u-sou,v-sov,0)]), None, approx_subgrid, kernel)[0]
    print("recomb+degrid:", deg_ap);
    print("recomb error: ", numpy.abs(deg_ap-deg))
    print("total error:  ", numpy.abs(deg_ap-actual))
    print("w theta^2:    ", w * theta**2)
    if show_subgrid:

        lam = xM_size / N
        uv_lower, uv_upper = coordinateBounds(xM_size)
        uv_lower = (uv_lower-1./xM_size/2)*lam
        uv_upper = (uv_upper+1./xM_size/2)*lam
        extent = (uv_lower+sou, uv_upper+sou,
                  uv_lower+sov, uv_upper+sov)
        
        fig = pylab.figure(figsize=(16,16))
        ax = fig.add_subplot(131)
        ax.imshow(approx_subgrid.real,extent=extent)
        
        #Fg = numpy.exp(2.j * numpy.pi * w * numpy.sqrt(1 - (coordinates(res) * theta)**2))
        Gn = numpy.abs(ifft(pswfs[W]**2))
        p = find_x_sorted_logsmooth(-coordinates(res), Gn, err_base[W])
        
        ov = 4
        print('effective theta:', theta*gc_x0)
        Fg = pad_mid(numpy.exp(2.j * numpy.pi * w * numpy.sqrt(1 - (coordinates(res*ov//2)*theta*gc_x0*2)**2)),
                     res*ov)
        Fn = test_pswf(W, ov)
        Gn = numpy.abs(ifft(Fg * Fn**2))
        #pylab.semilogy(numpy.abs(Fg*Fn**2))
        p = find_x_sorted_logsmooth(-coordinates(res*ov), Gn, err_base[W])
        print('p=', p)

        degrid_patch = lambda: patches.Rectangle((u-gc_support//2/N, v-gc_support//2/N),
                                                  gc_support/N, gc_support/N, fill=False)
        print(W, W2s_map[W])
        xA_size_ = xM_size - p * ov * res / yN_size * N #W2s_map[W] / yN_size * N
        xA_patch = lambda: patches.Rectangle((sou-xA_size_//2/N,sov-xA_size_//2/N),
                                             (xA_size_-1)/N, (xA_size_-1)/N, fill=False)
        ax.add_patch(degrid_patch())
        if xA_size_ > 0: ax.add_patch(xA_patch())
        ax2 = fig.add_subplot(132)
        im2 = ax2.imshow(numpy.log(numpy.abs(approx_subgrid - true_subgrid)) / numpy.log(10), extent=extent)
        ax2.add_patch(degrid_patch())
        if xA_size_ > 0: ax2.add_patch(xA_patch())
        fig.colorbar(im2, shrink=.4)
        ax3 = fig.add_subplot(133)
        AG = extract_mid(approx_subgrid - true_subgrid, int(xA_size_))
        print('subgrid area rmse', numpy.sqrt(numpy.mean(numpy.abs(AG)**2)))
        im3 = ax3.imshow(numpy.log(numpy.abs(AG)) / numpy.log(10)
                         , extent=extent)
        pylab.show(fig)

In [ ]:
import os.path
import h5py

out_prefix = "../../data/grid/T05b_"
@interact_manual
def export_test(test_path = "../../data/grid/T06_pswf_%s.in" % cfg_name):

    with h5py.File(out_prefix + "in.h5",'w') as f:
        f['pswf'] = numpy.fft.ifftshift(pswf)
        f['sepkern/kern'] = kernel
        for j0,j1 in itertools.product(range(nfacet), range(nfacet)):
            f["j0=%d/j1=%d/facet" % (j0,j1)] = numpy.fft.ifftshift(facet_2[j0,j1])
            for i0,i1 in itertools.product(range(nsubgrid), range(nsubgrid)):
                f['i0=%d/i1=%d/j0=%d/j1=%d/nmbf' % (i0,i1,j0,j1)] = \
                    numpy.fft.ifftshift(NMBF_NMBF[i0,i1,j0,j1])
        for i0,i1 in itertools.product(range(nsubgrid), range(nsubgrid)):
            #f["i0=%d/i1=%d/subgrid" % (i0,i1)] = numpy.fft.ifftshift(subgrid_2[i0, i1])
            f["i0=%d/i1=%d/approx" % (i0,i1)] = numpy.fft.ifftshift(make_approx_subgrid(i0, i1)[0])
        for sv in range(nsubgrid):
            for su in range(nsubgrid):
                # Write
                f['i0=%d/i1=%d/uvw' % (sv,su)] = uvws[sel_sg[sv,su]]
                f['i0=%d/i1=%d/uvw_subgrid' % (sv,su)] = uvw_sg[sv,su]
                f['i0=%d/i1=%d/vis' % (sv,su)] = deg[sel_sg[sv,su]]
                #f['i0=%d/i1=%d/vis_approx' % (sv,su)] = deg_ap

In [ ]:
import h5py

@interact_manual
def export_kernel(kernel_path = "../../data/grid/kernel_%d_%g.in" % (gc_support, gc_x0)):
    with h5py.File(kernel_path,'w') as f:
        f['sepkern/corr'] = numpy.fft.ifftshift(gc_pswf)
        f['sepkern/kern'] = kernel
        f['sepkern/x0'] = gc_x0